## 实验介绍

### 1.实验内容

本实验介绍基于用户的协同过滤算法的电影推荐系统。

### 2.实验目标

通过本实验掌握协同过滤算法。

### 3.实验知识点

* 协同过滤算法

### 4.实验环境

* python 3.6.5  
* CourseGrading在线实验环境

### 5.预备知识

* 初等数学知识  
* Linux命令基本操作  
* Python编程基础

## 准备工作
点击屏幕右上方的下载实验数据模块，选择下载movie_recommender.tgz到指定目录下，然后再依次选择点击上方的File->Open->Upload,上传刚才下载的数据集压缩包，再使用如下命令解压：

In [1]:
!tar -zxvf movie_recommender.tgz

movie_recommender/
movie_recommender/ratings.csv


In [7]:
!dir / a

 驱动器 D 中的卷没有标签。
 卷的序列号是 0C70-98D8

 D:\Python\PycharmProjects\Project1\Machine Learning\课件\Day 5 的目录

2022/11/13  16:40    <DIR>          .
2022/11/13  16:40    <DIR>          ..
2022/10/23  10:20    <DIR>          coll_filter
2022/10/23  10:21            10,953 coll_filter.ipynb
2022/11/13  15:54    <DIR>          movie_recommender
2022/11/13  16:40            23,284 movie_recommender.ipynb
               2 个文件         34,237 字节
               4 个目录 38,834,872,320 可用字节


## 【实验步骤】获取数据集 
本次实验使用公开数据集MovieLens提供的由600多个用户在近9000部电影上的评分构成的10万多条记录。数据存储在数据集目录的ratings.csv文件中，可直接调用。数据格式如下：

In [1]:
!cat movie_recommender / ratings.csv

'cat' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [8]:
import pandas as pd

data = pd.read_csv('movie_recommender/ratings.csv')
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


其中，userId为用户ID，每个用户对应唯一ID。movieId为电影ID，每个ID对应唯一一部电影。rating为某用户在某部电影上的评分，最高分为5分，最低分为0.5分。无缺失值。timestamp为用户评分时间戳，本次实验不使用该数据。实验中将原始数据划分为训练集和测试集，使用随机数将同一用户的数据随机的划分到测试集和训练集。

代码实现：

In [9]:
def get_dataset(self, filename, pivot=0.75):
    trainSet_len = 0
    testSet_len = 0
    for line in self.load_file(filename):
        user, movie, rating, timestamp = line.split(',')

        if random.random() < pivot:
            self.trainSet.setdefault(user, {})
            self.trainSet[user][movie] = rating
            trainSet_len += 1
        else:
            self.testSet.setdefault(user, {})
            self.testSet[user][movie] = rating
            testSet_len += 1
    print('划分训练集和测试集！')
    print('训练集个数： %s' % trainSet_len)
    print('测试集个数： %s' % testSet_len)

## 【实验步骤】建立用户电影矩阵模型 
协同过滤算法的输入数据通常表示为一个m*n的用户评价矩阵Matrix，m是用户数，n是电影数，Matrix[ij]表示第i个用户对第j个电影的评价：
![](movie_recommender/1_movie_recommender.png)
## 【实验步骤】发现兴趣相似的用户 
这一阶段，主要完成对目标用户最近邻居的查找，通过计算目标用户与其他用户之间的相似度，得到与目标用户最近的邻居集。度量用户间相似性：设N(u)为用户u喜欢的电影集合，N(v)为用户v喜欢的电影集合，将上一步中每行记录视为一个向量，那么u和v的相似度可通过以下进行计算：

    (a)采用Jaccard公式：
![](movie_recommender/2_movie_recommender.png)

    (b)余弦相似度计算：
![](movie_recommender/3_movie_recommender.png)

这里选择余弦公式进行相似度度量计算，假设目前共有4个用户(A、B、C、D)，5部电影(a、b、c、d、e)，用户与电影的关系如下图所示：
![](movie_recommender/4_movie_recommender.png)
![](movie_recommender/5_movie_recommender.png)
而这种方法的时间复杂度是O(|U| * |U|)，所以非常耗时。而且在上表中可以看到“用户-电影”表是一个稀疏矩阵，即很多时候N(U)∩N(V)=0，如果换一下思路，可以首先计算N(U)∩N(V)!=0的用户，然后再计算
![](movie_recommender/6_movie_recommender.png)
为此可以首先建立“电影-用户”的倒排表，对每部电影都保存电影到用户的列表：
![](movie_recommender/7_movie_recommender.png)
设稀疏矩阵C[u][v]=N(u)^N(v)，在倒排索引中假设用户u和用户v同时属于倒排索引中K部电影对应的用户列表，就有C[u][v]=K。例如上图所示只有电影a中同时出来了用户有A和用户B，则在矩阵中赋值为1：
![](movie_recommender/8_movie_recommender.png)
接着对N(U)∩N(V)!=0的用户进行相似度计算。
![](movie_recommender/9_movie_recommender.png)
到此，用户间的相似度计算就得到了，可以很直观的找到与目标用户兴趣相似的用户。

In [11]:
def calc_user_sim(self):
    #电影-用户
    movie_user = {}
    for user, movies in self.trainSet.items():
        for movie in movies:
            #遍历用户数据构建电影索引
            if movie not in movie_user:
                movie_user[movie] = set()
            movie_user[movie].add(user)

    self.movie_count = len(movie_user)
    print("电影总数：%d" % self.movie_count)
    #构建用户相似度矩阵第一步
    for movie, users in movie_user.items():
        for u in users:
            for v in users:
                if u == v:
                    continue
                #如果字典中该键值不存在，新增该键并设置默认值
                self.user_sim_matrix.setdefault(u, {})
                self.user_sim_matrix[u].setdefault(v, 0)
                self.user_sim_matrix[u][v] += 1
    #计算当前值非0的用户之间的相似度
    for u, related_users in self.user_sim_matrix.items():
        for v, count in related_users.items():
            self.user_sim_matrix[u][v] = count / math.sqrt(len(self.trainSet[u]) * len(self.trainSet[v]))


## 【实验步骤】产生推荐项目 
接下来，需要从矩阵中找到与目标用户最相似的K个用户，用集合S(u,K)表示，将S中用户喜欢的电影全部提取出来，并除去u已经喜欢的电影。对每个候选电影i，用户对它的感兴趣的程度用以下公式计算：
![](movie_recommender/10_movie_recommender.png)
（其中Rvi表示用户v对电影i的喜欢程度，此处举例全部为1，在电影评分时应该代入用户的评分）。

继续上面的例子，假设我们给A推荐电影，选取K=3，对用户A，电影c、e没有看过，因此可以将这两部电影推荐给用户A，根据UserCF算法用户A对物品c、e的兴趣分别计算p(A,c)和p(A,e)：
![](movie_recommender/11_movie_recommender.png)
所以用户A对电影c和e的喜欢程度可能一样，在真实的推荐系统中计算时考虑用户的评分，最后根据得分排序取前K个即为推荐电影。

代码实现：

In [12]:
 def recommend(self, user):
    K = self.n_sim_user  #寻找与user相似度最高的TopK
    N = self.n_rec_movie  #寻找与user关联度最高的TopN

    rank = {}
    #排除user已经看过的电影列表
    watched_movies = self.trainSet[user]
    #使用矩阵的第一列进行sort,以用户相关度进行比较
    #user_sim_matrix为用户相似度矩阵
    for v, wuv in sorted(self.user_sim_matrix[user].items(), key=itemgetter(1), reverse=True)[0:K]:
        for movie in self.trainSet[v]:
            if movie in watched_movies:
                continue
            #若该电影用户未看过，添加rank列表，计算关联值
            rank.setdefault(movie, 0)
            rank[movie] += wuv
    #根据user与电影的关联值排序，返回前N个电影。
    sim = sorted(rank.items(), key=itemgetter(1), reverse=True)[0:N]
    return sim

## 【实验步骤】推荐系统中准确率和召回率的理解 
推荐系统中的TopN推荐，它的预测准确率一般是通过准确率和召回率来进行评估的，那么我们就要理解，什么是准确率，什么是召回率！

    准确率，顾名思义，就是准确程度。通过正确数/总数得到。
    召回率，我们可以理解为找到的数目与总的需要我们找到的数目的比。

精确率是针对我们预测结果而言的，它表示的是预测为正的样本中有多少是真正的正样本。
而召回率是针对我们原来的样本而言的，它表示的是样本中的正例有多少被预测正确了。

代码实现：

In [13]:
# 产生推荐并通过准确率、召回率和覆盖率进行评估
def evaluate(self):
    print("开始评估 ...")
    N = self.n_rec_movie
    # 准确率和召回率
    hit = 0
    rec_count = 0
    test_count = 0
    # 覆盖率
    all_rec_movies = set()

    for i, user, in enumerate(self.trainSet):
        test_movies = self.testSet.get(user, {})
        rec_movies = self.recommend(user)
        #数据导出配置，无意义
        result = pd.DataFrame(rec_movies, columns=['movieid', 'score'])
        result['userid'] = user
        result = result[['userid', 'movieid', 'score']]
        result.set_index(['userid', 'movieid'])

        if i == 0:
            result.to_csv('result.csv', index=False)
        else:
            result.to_csv('result.csv', mode='a', header=False, index=False)
        #命中计算
        for movie, w in rec_movies:
            #如果推荐的电影在该用户的给出的电影集中
            if movie in test_movies:
                hit += 1
            all_rec_movies.add(movie)
        #计算准确率总数
        rec_count += N
        # 计算召回率总数
        test_count += len(test_movies)

    precision = hit / (1.0 * rec_count)
    recall = hit / (1.0 * test_count)
    coverage = len(all_rec_movies) / (1.0 * self.movie_count)
    print('准确度：%.4f\t召回率：%.4f\t覆盖率：%.4f' % (precision, recall, coverage))

## 【实验步骤】代码实现 

In [16]:
import random
import pandas as pd
import math
from operator import itemgetter


class UserBasedCF():
    def __init__(self):
        self.n_sim_user = 20
        self.n_rec_movie = 10
        #此处考虑是否可以换为crossvalidation
        self.trainSet = {}
        self.testSet = {}

        #用户相似度矩阵
        self.user_sim_matrix = {}  #dic
        self.movie_count = 0

    def get_dataset(self, filename, pivot=0.75):
        trainSet_len = 0
        testSet_len = 0
        for line in self.load_file(filename):
            user, movie, rating, timestamp = line.split(',')
            if random.random() < pivot:
                self.trainSet.setdefault(user, {})
                self.trainSet[user][movie] = rating
                trainSet_len += 1
            else:
                self.testSet.setdefault(user, {})
                self.testSet[user][movie] = rating
                testSet_len += 1
        print('划分训练集和测试集！')
        print('训练集个数： %s' % trainSet_len)
        print('测试集个数： %s' % testSet_len)

    # 读文件，返回文件的每一行
    def load_file(self, filename):
        with open(filename, 'r') as f:
            for i, line in enumerate(f):
                if i == 0:  # 去掉文件第一行的title
                    continue
                yield line.strip('\r\n')
        print('加载文件 %s 成功!' % filename)

    def calc_user_sim(self):
        #电影-用户
        movie_user = {}
        for user, movies in self.trainSet.items():
            for movie in movies:
                if movie not in movie_user:
                    movie_user[movie] = set()
                movie_user[movie].add(user)

        self.movie_count = len(movie_user)
        print("电影总数：%d" % self.movie_count)
        #构建用户相似度矩阵第一步
        for movie, users in movie_user.items():
            for u in users:
                for v in users:
                    if u == v:
                        continue
                    #如果字典中该键值不存在，新增该键并设置默认值
                    self.user_sim_matrix.setdefault(u, {})
                    self.user_sim_matrix[u].setdefault(v, 0)
                    self.user_sim_matrix[u][v] += 1
        #计算当前值非0的用户之间的相似度
        for u, related_users in self.user_sim_matrix.items():
            for v, count in related_users.items():
                self.user_sim_matrix[u][v] = count / math.sqrt(len(self.trainSet[u]) * len(self.trainSet[v]))

    def recommend(self, user):
        K = self.n_sim_user
        N = self.n_rec_movie

        rank = {}
        watched_movies = self.trainSet[user]
        #使用矩阵的第一列进行sort,以用户相关度进行比较
        for v, wuv in sorted(self.user_sim_matrix[user].items(), key=itemgetter(1), reverse=True)[0:K]:
            for movie in self.trainSet[v]:
                if movie in watched_movies:
                    continue
                rank.setdefault(movie, 0)
                rank[movie] += wuv
        sim = sorted(rank.items(), key=itemgetter(1), reverse=True)[0:N]
        return sim

    # 产生推荐并通过准确率、召回率和覆盖率进行评估
    def evaluate(self):
        print("开始评估 ...")
        N = self.n_rec_movie
        # 准确率和召回率
        hit = 0
        rec_count = 0
        test_count = 0
        # 覆盖率
        all_rec_movies = set()

        for i, user, in enumerate(self.trainSet):
            test_movies = self.testSet.get(user, {})
            rec_movies = self.recommend(user)
            result = pd.DataFrame(rec_movies, columns=['movieid', 'score'])
            result['userid'] = user
            result = result[['userid', 'movieid', 'score']]
            result.set_index(['userid', 'movieid'])
            if i == 0:
                result.to_csv('result.csv', index=False)
            else:
                result.to_csv('result.csv', mode='a', header=False, index=False)
            for movie, w in rec_movies:
                if movie in test_movies:
                    hit += 1
                all_rec_movies.add(movie)
            rec_count += N
            test_count += len(test_movies)

        precision = hit / (1.0 * rec_count)
        recall = hit / (1.0 * test_count)
        coverage = len(all_rec_movies) / (1.0 * self.movie_count)
        print('准确度：%.4f\t召回率：%.4f\t覆盖率：%.4f' % (precision, recall, coverage))


if __name__ == '__main__':
    rating_file = 'movie_recommender/ratings.csv'
    userCF = UserBasedCF()
    userCF.get_dataset(rating_file)
    userCF.calc_user_sim()
    userCF.evaluate()

加载文件 movie_recommender/ratings.csv 成功!
划分训练集和测试集！
训练集个数： 75632
测试集个数： 25204
电影总数：8773
开始评估 ...
准确度：0.2997	召回率：0.0725	覆盖率：0.0423


可以看到，在覆盖率为4.1%的情况下，准确率为29.8%，召回率7%。以及对用户ID为1的用户推荐的10个电影。

##  实验总结 
本实验介绍基于用户的协同过滤算法的电影推荐系统。

学生通过该实验需达到以下目标：

    能够实现基于用户的协同过滤算法
    能够实现推荐系统

## 参考文献与延伸阅读

### 参考资料:

1.哈林顿，李锐. 机器学习实战 : Machine learning in action[M]. 人民邮电出版社, 2013.  
2.周志华. 机器学习:Machine learning[M]. 清华大学出版社, 2016.

### 延伸阅读

1.李航. 统计学习方法[M]. 清华大学出版社, 2012.